# Homesite Quote Conversion <a id='goback'> </a>
![image.png](Homesite_image.png) 

**Background** <br>
This dataset represents the activity of a large number of customers who are interested in buying policies from Homesite. Each QuoteNumber corresponds to a potential customer and the QuoteConversion_Flag indicates whether the customer purchased a policy.
<br>
The provided features are anonymized and provide a rich representation of the prospective customer and policy. They include specific coverage information, sales information, personal information, property information, and geographic information. Your task is to predict QuoteConversion_Flag for each QuoteNumber in the test set.

**References** <br>
    - [Pandas Library](https://pandas.pydata.org/docs/index.html) <br>
    - [SkLearn Library](https://scikit-learn.org/stable/) <br>
    - [SMOTE API](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTE.html) <br>
    - [Stacking Library](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html) <br>
    - [Kaggle Dataset](https://www.kaggle.com/competitions/homesite-quote-conversion/data?select=test.csv.zip) <br>

**Notebook Index**
- [Data Import](#dataimport)
- [Data Preprocessing](#preprocess)
    - [One Hot Encoding : Categorical Values](#ohe)
    - [Handling Imbalanced Datasets](#imbalance)
- [Effect of SMOTE on a classification model ](#smote)
- [Accuracy Comparison](#acccompare)
    - [Multiple Model Development](#multiple)
    - [Ensemble Method : Stacking](#ensemble)
    - [Accuracy Report](#acc)
- [Kaggle Submission](#kag)
    - [Multiple Model Development with HyperParameter Tuning](#multiple2)
    - [Ensemble Method : Stacking](#ensemble2)
    - [Kaggle Report](#kag2)

In [ ]:
#Libraries
# Data Manipulation
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sn

# Imbalanced Data
from imblearn.over_sampling import RandomOverSampler,SMOTE
from imblearn.under_sampling import RandomUnderSampler,NearMiss

# Machine Learning 
import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score,roc_curve,mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Hyper Parameter Tuning
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

# Stacking
from vecstack import stacking

import warnings
warnings.filterwarnings("ignore")

# Function to evaluate the model results
def evaluate_model(model,xtrain,xtest,ytrain,ytest):
    model.fit(xtrain,ytrain)
    ypred=model.predict(xtest)
    
    print(model)
    print("Testing Accuracy Score: ",accuracy_score(ytest,ypred))
    print()
    print("Classification Report: \n",classification_report(ytest,ypred))
    print()

# SMOTEing on Training data only and Accuracy with YTest
def evaluate_modeL_smote(model,X,Y,k):
    #If k=0 , no oversmapling required
    
    Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.3)
    
    # SMOTEingonly on Train part of Train_df
    if k!=0: 
        sm=SMOTE(sampling_strategy=k)
        SMOTE_X,SMOTE_Y=sm.fit_resample(Xtrain,Ytrain)
    else:
        SMOTE_X=Xtrain
        SMOTE_Y=Ytrain
    
    # Fitting using SMOTE_X,SMOTE_Y
    model.fit(SMOTE_X,SMOTE_Y)
    YPred=model.predict(Xtest)
    
    print(model)
    print("\nSMOTE sampling_strategy:",k)
    print("Testing Accuracy Score: ",accuracy_score(Ytest,YPred))

## Data Import <a id='dataimport'> </a>

In [ ]:
TrainingFile_loc=r'Data/RevisedHomesiteTrain1.csv'
Train_df=pd.read_csv(TrainingFile_loc)

TestingFile_loc=r'Data/RevisedHomesiteTest1.csv'
Test_df=pd.read_csv(TestingFile_loc)

Train_df.head()

In [ ]:
Test_df.head()

`[Go Back to Index](#goback)

## Data Preprocessing <a id="preprocess"> </a>
One Hot Encoding for Categorical Datapoints <a id='ohe'> </a>

In [ ]:
# Issue in Encoding GrographicalField64 
# Drop
Test_df.drop(["GeographicField64_CA","GeographicField64_IL","GeographicField64_NJ","GeographicField64_TX"],axis=1,inplace=True)

# Perform OneHotEncoding to Code location IL,NJ,TX,CA
from sklearn.preprocessing import OneHotEncoder
GeoFieldEnc=OneHotEncoder(sparse=False,handle_unknown='ignore')
EncodedValues=pd.DataFrame(GeoFieldEnc.fit_transform(Test_df[['GeographicField64']]),
                           columns=GeoFieldEnc.get_feature_names_out())

Test_df=pd.concat([Test_df,EncodedValues],axis=1)
Test_df
#Dropping a categorical column 
Test_df.drop("GeographicField64",axis=1,inplace=True)

In [ ]:
Test_df.head()

Successfully encoded GrographicalField64!

Drop any Row Duplicates 

In [ ]:
Train_df.shape

In [ ]:
Train_df.drop_duplicates()
Train_df.shape

In [ ]:
Test_df.shape

No Row Duplicates found

Search for any null values in the datasets

In [ ]:
Train_df.isnull().sum().sort_values(ascending=False)

No Null values in the dataset

In [ ]:
X=Train_df.drop("QuoteConversion_Flag",axis=1)
Y=Train_df["QuoteConversion_Flag"]

Y.value_counts(), Y.value_counts().plot.pie()

`[Go Back to Index](#goback)

### Handling Imbalanced Datasets <a id='imbalance'> </a>
- Random Under Sampling
- Random Over Sampling
- **SMOTE**: Synthetic Minority Oversampling Technique

In [ ]:
#Random Under-Sampling
# rus=RandomUnderSampler(sampling_strategy=1)
# X_rus,Y_rus=rus.fit_resample(X,Y)
# Y_rus.value_counts(), Y_rus.value_counts().plot.pie()

# # Near Miss Under Sampling 
# # Three Methods : 
# Nearmiss1: K nearest neighbours having smallest avg distance,
# Nearmiss K neighbours farthest having smallest avg distance2,nearmiss 3
# nmus=NearMiss(sampling_strategy=1)
# X_nmus,Y_nmus=nmus.fit_resample(X,Y)
# Y_nmus.value_counts().plot.pie()

In [ ]:
#Random Over-Sampling
# rus=RandomOverSampler(sampling_strategy=1)
# X_ros,Y_ros=rus.fit_resample(X,Y)
# Y_ros.value_counts(), Y_ros.value_counts().plot.pie()

#Synethetic Minority Oversampling Technique : SMOTE
sm=SMOTE(sampling_strategy=1)
X_sm,Y_sm=sm.fit_resample(X,Y)
Y_sm.value_counts(), Y_sm.value_counts().plot.pie()

`[Go Back to Index](#goback)

## Effect of SMOTE on a classification model <a id='smote'> </a>

In [ ]:
# Model1 : Decision Tree
M1=DecisionTreeClassifier()
#No SMOTE
evaluate_modeL_smote(M1,X,Y,0)

In [ ]:
# Model1 : Decision Tree
M12=DecisionTreeClassifier()
evaluate_modeL_smote(M12,X,Y,1)

In [ ]:
M13=DecisionTreeClassifier()
evaluate_modeL_smote(M13,X,Y,0.8)

In [ ]:
M14=DecisionTreeClassifier()
evaluate_modeL_smote(M14,X,Y,0.5)

`[Go Back to Index](#goback)

## Accuracy Comparison <a id='acccompare'> </a>
### Multiple Model Development <a id='multiple'> </a>
- We use a sampling_strategy of 1 as it provides us with equal instances of both classes
- Developing 5 Models : Decision Tree, Random Forest, Support Vector Machine, KNN Classfier, Neural Nets MPT

In [ ]:
%%time
#With Test_Train Split function
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.3)

sm=SMOTE(sampling_strategy=1)
SMOTE_X,SMOTE_Y=sm.fit_resample(Xtrain,Ytrain)

# Model1 : Decision Tree
M1=DecisionTreeClassifier()
evaluate_model(M1,SMOTE_X,Xtest,SMOTE_Y,Ytest)

In [ ]:
%%time
# Model2 : Random Forest Classifier
M2=RandomForestClassifier()
evaluate_model(M2,SMOTE_X,Xtest,SMOTE_Y,Ytest)

In [ ]:
%%time
# Model3 : Support Vector Machine
M3=LinearSVC()
evaluate_model(M3,SMOTE_X,Xtest,SMOTE_Y,Ytest)

In [ ]:
%%time
# Model4 : KNN Classifier
M4=KNeighborsClassifier()
evaluate_model(M4,SMOTE_X,Xtest,SMOTE_Y,Ytest)

In [ ]:
# Model5 : Neural Nets
M5=MLPClassifier()
evaluate_model(M5,SMOTE_X,Xtest,SMOTE_Y,Ytest)

`[Go Back to Index](#goback)

### Ensemble Method <a id='ensemble'> </a>
Stacking 5 models and taking RandomForestClassifier as Top Level Model

In [ ]:
%%time
# Stacking 5 models using just Train_df

print("__________________________________________STACKING with TestTrainSplit______________________________________________")

# models=[M1,M2,M3,M4,M5]
models=[DecisionTreeClassifier(),RandomForestClassifier(),LinearSVC(),KNeighborsClassifier(),MLPClassifier()]

S_train,S_Test=stacking(models,
                        SMOTE_X,SMOTE_Y,Xtest,
                        shuffle=True,
                        regression=False,
                        metric=accuracy_score,
                        verbose=2
                       )

FinalModel=RandomForestClassifier()
FinalModel.fit(S_train,SMOTE_Y)
ypred_train=FinalModel.predict(S_train)
ypred_test=FinalModel.predict(S_Test)

print("Training Accuracy:",accuracy_score(SMOTE_Y,ypred_train))
print("Testing Accuracy:",accuracy_score(Ytest,ypred_test))
    
print("Mean Square Error Train:",mean_squared_error(SMOTE_Y,ypred_train))
print("Mean Square Error Test:",mean_squared_error(Ytest,ypred_test))

`[Go Back to Index](#goback)

## Accuracy Comparison <a id='acc'> </a>
Using train_test_split on Train_df

|S No.| Model Name | Type of Model | SMOTE strategy | HyperParameter Tuning | Accuracy  |
|:-:| :---- | :---: | :---: |:---: |:---: |
|1.| Decision Tree M1 | Individual |1|None| 0.87 |
|2.| Random Forest Classifier M2 | Individual |1|None| <span style="color:red">0.89</span> |
|3.| LinearSVC Classifier M3 | Individual |1|None| 0.34 |
|4.| KNeighborsClassifier M4 | Individual |1|None| 0.57 |
|5.| MLP Classifier M5 | Individual |1|None| 0.80 |
| | | | | | |
|6.| **RandomForestClassifier** | **Stacked** |**1**|**None**| **0.89** |


`[Go Back to Index](#goback)

## Kaggle Submission <a id='kag'> </a>
### Multiple Model Development <a id='multiple2'> </a>
- Training models using SMOTEd Training data: X_sm, Y_sm <br>
- Testing models using Test_df <br>
- Sampling_strategy of 1 has been used for X_sm,Y_sm
- Developing 5 Models : Decision Tree, Random Forest, Support Vector Machine, KNN Classfier, Neural Nets MPT
- Hyper Parameter Tuning using RandomizedSeachCV method

In [ ]:
%%time
# Decision Tree
M6=DecisionTreeClassifier()
# M6.fit(X_sm,Y_sm)

# RandomCV Hyper Parameter Tuning
param6={'criterion':['gini', 'entropy'],
        'splitter':['best','random'],
       'max_depth':range(1,20,1)}
rcv=RandomizedSearchCV(M6,param6,verbose=2)
rcv.fit(X_sm,Y_sm)
print("Best set of parameters for dt: ",rcv.best_params_,
      "\nBest Score : ",rcv.best_score_)
parambest=rcv.best_params_

M6=DecisionTreeClassifier(**parambest)
M6.fit(X_sm,Y_sm)
Output6=M6.predict_proba(Test_df)

# Export as CSV
sub=pd.read_csv("sample_submission.csv")
output=pd.concat([sub["QuoteNumber"],pd.DataFrame(Output6[:,1],columns=["QuoteConversion_Flag"])],axis=1)
output.to_csv("M6_KaggleSubmissionAssignment3.csv",index=False)

In [ ]:
%%time
# Random Forest
M7=RandomForestClassifier()
# M7.fit(X_sm,Y_sm)

# RandomCV Hyper Parameter Tuning
param7={'n_estimators':range(10,200,2),
        'criterion':['gini', 'entropy'],
       'max_depth':range(1,20,2)}
rcv7=RandomizedSearchCV(M7,param7,verbose=2)
rcv7.fit(X_sm,Y_sm)
print("Best set of parameters for dt: ",rcv7.best_params_,
      "\nBest Score : ",rcv7.best_score_)
parambest=rcv7.best_params_

M7=RandomForestClassifier(**parambest)
M7.fit(X_sm,Y_sm)
Output7=M7.predict_proba(Test_df)

# Export as CSV
sub=pd.read_csv("sample_submission.csv")
output=pd.concat([sub["QuoteNumber"],pd.DataFrame(Output7[:,1],columns=["QuoteConversion_Flag"])],axis=1)
output.to_csv("M7_KaggleSubmissionAssignment3.csv",index=False)

In [ ]:
%%time
# Linear SVM
M8=LinearSVC()
# M8.fit(X_sm,Y_sm)

# RandomCV Hyper Parameter Tuning
param8={'max_iter':range(200,2000,200)}

rcv8=RandomizedSearchCV(M8,param8,verbose=2)
rcv8.fit(X_sm,Y_sm)
print("Best set of parameters for dt: ",rcv8.best_params_,
      "\nBest Score : ",rcv8.best_score_)
parambest=rcv8.best_params_

M8=LinearSVC(**parambest)
M8.fit(X_sm,Y_sm)
Output8=M8.predict(Test_df)

# Export as CSV
sub=pd.read_csv("sample_submission.csv")
output=pd.concat([sub["QuoteNumber"],pd.DataFrame(Output8,columns=["QuoteConversion_Flag"])],axis=1)
output.to_csv("M8_KaggleSubmissionAssignment3.csv",index=False)

In [ ]:
%%time
# KNN Classifier
M9=KNeighborsClassifier()
M9.fit(X_sm,Y_sm)

# RandomCV Hyper Parameter Tuning
param9={'n_neighbors':range(3,13,2),
        'weights':['uniform', 'distance'],
       'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
rcv9=RandomizedSearchCV(M9,param9,verbose=2)
rcv9.fit(X_sm,Y_sm)
print("Best set of parameters for dt: ",rcv9.best_params_,
      "\nBest Score : ",rcv9.best_score_)
parambest=rcv9.best_params_

M9=KNeighborsClassifier(**parambest)
M9.fit(X_sm,Y_sm)
Output9=M9.predict_proba(Test_df)

# Export as CSV
sub=pd.read_csv("sample_submission.csv")
output=pd.concat([sub["QuoteNumber"],pd.DataFrame(Output9[:,1],columns=["QuoteConversion_Flag"])],axis=1)
output.to_csv("M9_KaggleSubmissionAssignment3.csv",index=False)

In [ ]:
%%time
# MLP Classifier
M10=MLPClassifier()
# M10.fit(X_sm,Y_sm)

# RandomCV Hyper Parameter Tuning
param10={'hidden_layer_sizes':[(10,),(20,),(40,),(60,),(80,),(100,)],
        'activation':['identity', 'logistic','tanh','relu'],
        'solver':['sgd','adam'],
        'learning_rate':['constant', 'invscaling','adaptive']}
rcv10=RandomizedSearchCV(M10,param10,verbose=2)
rcv10.fit(X_sm,Y_sm)
print("Best set of parameters for dt: ",rcv10.best_params_,
      "\nBest Score : ",rcv10.best_score_)
parambest=rcv10.best_params_

M10=MLPClassifier(**parambest)
M10.fit(X_sm,Y_sm)
Output10=M10.predict_proba(Test_df)

# Export as CSV
sub=pd.read_csv("sample_submission.csv")
output=pd.concat([sub["QuoteNumber"],pd.DataFrame(Output10[:,1],columns=["QuoteConversion_Flag"])],axis=1)
output.to_csv("M10_KaggleSubmissionAssignment3.csv",index=False)

`[Go Back to Index](#goback)

### Ensemble Method <a id='ensemble2'> </a>
Stacking 5 models and taking RandomForestClassifier and DecisionTree as Top Level Model

In [ ]:
# Stacking
# Level 0 Models
models=[RandomForestClassifier(),
        DecisionTreeClassifier(),
        KNeighborsClassifier(),
        LinearSVC(),
        MLPClassifier()]

# We can also use our hyper parameter tuned individual modesl as input for stacking
# models=[M6,M7,M8,M9,M10]
# models=[M6,M7,M10]

S_Train_hp,S_Test_hp=stacking(models,
                             X_sm,Y_sm,Test_df,
                             shuffle=True,
                             stratified=True,
                             n_folds=4,
                             regression=False,
                             metric=accuracy_score,
                             verbose=2)
print('Stacking Complete!')

In [ ]:
%%time
# Decision Tree Level 1 Model
FinalModel_dt_hp=DecisionTreeClassifier()
FinalModel_dt_hp.fit(S_Train_hp,Y_sm)

# RandomCV Hyper Parameter Tuning
final_param={'criterion':['gini', 'entropy'],
             'max_depth':range(1,20,2)}

rcv=RandomizedSearchCV(FinalModel_dt_hp,final_param,verbose=2)
rcv.fit(S_Train_hp,Y_sm)
print("Best set of parameters for dt: ",rcv.best_params_,
      "\nBest Score : ",rcv.best_score_)
parambest=rcv.best_params_

FinalModel_dt_hp=DecisionTreeClassifier(**parambest)
FinalModel_dt_hp.fit(S_Train_hp,Y_sm)

OutputFinal=FinalModel_dt_hp.predict_proba(S_Test_hp)

# Export as CSV
sub=pd.read_csv("sample_submission.csv")
output=pd.concat([sub["QuoteNumber"],pd.DataFrame(OutputFinal[:,1],columns=["QuoteConversion_Flag"])],axis=1)
output.to_csv("Final_dt_KaggleSubmissionAssignment3.csv",index=False)

`[Go Back to Index](#goback)

In [ ]:
%%time
# Random Forest Level 1 Model
FinalModel_rf_hp=RandomForestClassifier()
FinalModel_rf_hp.fit(S_Train_hp,Y_sm)

# RandomCV Hyper Parameter Tuning
final_param={'n_estimators':range(10,200,10),
             'criterion':['gini', 'entropy'],
             'max_depth':range(1,20,2)}

rcv=RandomizedSearchCV(FinalModel_rf_hp,final_param,verbose=2)
rcv.fit(S_Train_hp,Y_sm)
print("Best set of parameters for dt: ",rcv.best_params_,
      "\nBest Score : ",rcv.best_score_)
parambest=rcv.best_params_

FinalModel_rf_hp=RandomForestClassifier(**parambest)
FinalModel_rf_hp.fit(S_Train_hp,Y_sm)

OutputFinal=FinalModel_rf_hp.predict_proba(S_Test_hp)

# Export as CSV
sub=pd.read_csv("sample_submission.csv")
output=pd.concat([sub["QuoteNumber"],pd.DataFrame(OutputFinal[:,1],columns=["QuoteConversion_Flag"])],axis=1)
output.to_csv("Final_rf_KaggleSubmissionAssignment3.csv",index=False)

`[Go Back to Index](#goback)

### Kaggle Score Comparison <a id='kag2'> </a>
Final Kaggle Scores for each Individual Hyperparameter tuned model and Stacked HyperParameter tuned model

| Model Name | Type of Model |SMOTE strategy|HyperParameter Tuning| Kaggle Score |
| :---: | :---: | :---: |:---: |:---: |
| Decision Tree M6 | Individual | 1 |RandomizedSearchCV| 0.90 |
| Random Forest M7 | Individual | 1 |RandomizedSearchCV| <span style="color:red"> 0.94 </span>|
| Linear SVC M8 | Individual | 1 |RandomizedSearchCV| 0.64 |
| KNN Classifier M9 | Individual |1|RandomizedSearchCV| 0.50 |
| MLP Classifier M10 | Individual |1|RandomizedSearchCV| 0.68 |
| | | | |
| **Decision Tree** | Stacked |1|RandomizedSearchCV| **0.85** |
| **Random Forest** | Stacked |1|RandomizedSearchCV| **0.85** |


`[Go Back to Index](#goback)